# Memory
## Memory 是什麼？
Memory 讓 LangGraph 能夠記住對話歷史和 graph 的執行狀態，實現：

- 狀態持久化：保存每次執行的 state
- 對話延續：透過 thread_id 區分不同對話
- 歷史追蹤：查看 graph 在每個節點的狀態變化
- Time Travel：回到過去任意時間點繼續執行

## Checkpointer
LangGraph 使用 Checkpointer 來管理 Memory。每次 graph 執行完一個節點，就會保存當下的 state。

# 短期記憶
## MemorySaver（記憶體儲存）
MemorySaver 是 LangGraph 提供的 最簡單 checkpoint 實作，適合用於 本地測試、教學、開發階段。
它的功能是：
> 在每次 graph 執行結束後，將 state 儲存在記憶體中，並以 thread_id 作為索引

### 基本範例

In [1]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from typing import TypedDict

class AgentState(TypedDict):
    messages: list[str]
    count: int

def process_node(state: AgentState):
    new_message = f"處理第 {state['count']} 次"
    return {
        "messages": state["messages"] + [new_message],
        "count": state["count"] + 1
    }

# 建立 graph
workflow = StateGraph(AgentState)
workflow.add_node("process", process_node)
workflow.set_entry_point("process")
workflow.set_finish_point("process")

# 使用 MemorySaver
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "conversation_1"}}

說明：

- thread_id：對話識別碼，相同 thread_id 會共享狀態
- MemorySaver 只存在記憶體中，程式重啟後會消失

In [ ]:
state = graph.invoke({"messages": [], "count": 1}, config)
print(state)
while True:
    state = graph.invoke(state, config)
    print(state)

    if state["count"] > 3:
        break

{'messages': ['處理第 1 次'], 'count': 2}
{'messages': ['處理第 1 次', '處理第 2 次'], 'count': 3}
{'messages': ['處理第 1 次', '處理第 2 次', '處理第 3 次'], 'count': 4}


### 多對話管理

In [3]:
# 第一個對話
config_1 = {"configurable": {"thread_id": "user_alice"}}
state_1 = graph.invoke({"messages": [], "count": 1}, config_1)
print(f"Alice: {state_1}")

# 第二個對話
config_2 = {"configurable": {"thread_id": "user_bob"}}
state_2 = graph.invoke({"messages": [], "count": 10}, config_2)
print(f"Bob: {state_2}")

# Alice 繼續對話
state_1 = graph.invoke(state_1, config_1)
print(f"Alice 繼續: {state_1}")

# Bob 繼續對話
state_2 = graph.invoke(state_2, config_2)
print(f"Bob 繼續: {state_2}")

Alice: {'messages': ['處理第 1 次'], 'count': 2}
Bob: {'messages': ['處理第 10 次'], 'count': 11}
Alice 繼續: {'messages': ['處理第 1 次', '處理第 2 次'], 'count': 3}
Bob 繼續: {'messages': ['處理第 10 次', '處理第 11 次'], 'count': 12}


說明：

- 不同 thread_id 的對話狀態完全獨立
- 可以同時管理多個使用者的對話

### 查看歷史狀態

In [5]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "conversation_1"}}

# 執行多次
for i in range(3):
    state = graph.invoke({"messages": [], "count": i+1}, config)
    print(f"第 {i+1} 次執行: {state}")

# 查看所有 checkpoint
print("\n=== 查看歷史記錄 ===")
history = list(checkpointer.list(config))
print(f"共有 {len(history)} 個 checkpoint")

for i, checkpoint_tuple in enumerate(history):
    print(f"\nCheckpoint {i+1}:")
    print(f"  Config: {checkpoint_tuple.config}")
    print(f"  State: {checkpoint_tuple.checkpoint}")
    print(f"  Metadata: {checkpoint_tuple.metadata}")

第 1 次執行: {'messages': ['處理第 1 次'], 'count': 2}
第 2 次執行: {'messages': ['處理第 2 次'], 'count': 3}
第 3 次執行: {'messages': ['處理第 3 次'], 'count': 4}

=== 查看歷史記錄 ===
共有 9 個 checkpoint

Checkpoint 1:
  Config: {'configurable': {'thread_id': 'conversation_1', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0024-c506-6883-8007-b7f03066778c'}}
  State: {'v': 4, 'ts': '2025-12-23T13:21:32.337984+00:00', 'id': '1f0e0024-c506-6883-8007-b7f03066778c', 'channel_versions': {'__start__': '00000000000000000000000000000008.0.32898029730307543', 'messages': '00000000000000000000000000000009.0.5535717744752875', 'count': '00000000000000000000000000000009.0.5535717744752875', 'branch:to:process': '00000000000000000000000000000009.0.5535717744752875'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000007.0.3054610256736364'}, 'process': {'branch:to:process': '00000000000000000000000000000008.0.32898029730307543'}}, 'updated_channels': ['count', 'messages'], 'channel_values'

執行 3 次,每次執行會產生 3 個 checkpoint:
1. 初始 checkpoint (輸入階段)
2. 節點執行中 checkpoint (process 節點執行前)
3. 完成 checkpoint (process 節點執行後)

Checkpoint 結構：

In [ ]:
{
  'v': 4,  # checkpoint 版本號
  'ts': '2025-12-23T13:21:32.337984+00:00',  # 時間戳
  'id': '1f0e0024-c506-6883-8007-b7f03066778c',  # 唯一 ID
  
  # 各個通道(channel)的版本號
  'channel_versions': {
    '__start__': '...',      # 起始節點版本
    'messages': '...',       # messages 狀態版本
    'count': '...',          # count 狀態版本
    'branch:to:process': '...'  # 分支版本
  },
  
  # 哪些節點看過哪些版本(用於追蹤依賴)
  'versions_seen': {
    '__input__': {},
    '__start__': {...},
    'process': {...}
  },
  
  # 本次更新的通道
  'updated_channels': ['count', 'messages'],
  
  # 當前狀態值
  'channel_values': {
    'messages': ['處理第 3 次'],
    'count': 4
  }
}

Metadata 說明：

In [ ]:
'metadata': {
  'source': 'loop',  # 來源:循環執行
  'step': 7,         # 第 7 步(每次執行有多個步驟)
  'parents': {}      # 父級 checkpoint
}

### 取得特定時間點的狀態

In [7]:
# 取得最新的 checkpoint
latest = checkpointer.get(config)
if latest:
    print(f"最新狀態: {latest}")
    # latest 本身就是 checkpoint 字典
    channel_values = latest.get('channel_values', {})
    print(f"  Messages: {channel_values.get('messages', [])}")
    print(f"  Count: {channel_values.get('count', 0)}")

# 取得所有 checkpoint 並選擇特定的
history = list(checkpointer.list(config))
if len(history) >= 2:
    second_checkpoint_tuple = history[1]
    second_checkpoint = second_checkpoint_tuple.checkpoint
    print(f"\n第二個狀態: {second_checkpoint}")
    channel_values = second_checkpoint.get('channel_values', {})
    print(f"  Messages: {channel_values.get('messages', [])}")
    print(f"  Count: {channel_values.get('count', 0)}")

最新狀態: {'v': 4, 'ts': '2025-12-23T13:21:32.337984+00:00', 'id': '1f0e0024-c506-6883-8007-b7f03066778c', 'channel_versions': {'__start__': '00000000000000000000000000000008.0.32898029730307543', 'messages': '00000000000000000000000000000009.0.5535717744752875', 'count': '00000000000000000000000000000009.0.5535717744752875', 'branch:to:process': '00000000000000000000000000000009.0.5535717744752875'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000007.0.3054610256736364'}, 'process': {'branch:to:process': '00000000000000000000000000000008.0.32898029730307543'}}, 'updated_channels': ['count', 'messages'], 'channel_values': {'messages': ['處理第 3 次'], 'count': 4}}
  Messages: ['處理第 3 次']
  Count: 4

第二個狀態: {'v': 4, 'ts': '2025-12-23T13:21:32.336985+00:00', 'id': '1f0e0024-c504-617d-8006-4d11efc541c9', 'channel_versions': {'__start__': '00000000000000000000000000000008.0.32898029730307543', 'messages': '00000000000000000000000000000008.0.328980297303

### 實際應用：聊天機器人

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from typing import TypedDict

class ChatState(TypedDict):
    messages: list[dict]
    user_name: str

def chat_node(state: ChatState):
    user_msg = state["messages"][-1]["content"]
    bot_reply = f"你好 {state['user_name']}！你說：{user_msg}"
    
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": bot_reply}
        ]
    }

# 建立 chatbot
workflow = StateGraph(ChatState)
workflow.add_node("chat", chat_node)
workflow.set_entry_point("chat")
workflow.add_edge("chat", END)

checkpointer = MemorySaver()
chatbot = workflow.compile(checkpointer=checkpointer)

# 使用範例
config = {"configurable": {"thread_id": "user_123"}}

# 第一輪對話
state = chatbot.invoke({
    "messages": [{"role": "user", "content": "你好"}],
    "user_name": "小明"
}, config)
print(state["messages"][-1]["content"])

# 第二輪對話
state = chatbot.invoke({
    "messages": [{"role": "user", "content": "天氣如何？"}],
    "user_name": "小明"
}, config)

print(state["messages"][-1]["content"])
print(f"\n對話記錄共 {len(state['messages'])} 則")

你好 小明！你說：你好
你好 小明！你說：天氣如何？

對話記錄共 2 則


# 持久化儲存（Persistent Checkpointers）
## MemorySaver 的限制
MemorySaver 雖然簡單易用，但有以下限制：

1. 不持久化：程式重啟後資料消失
2. 記憶體限制：對話多了會佔用大量記憶體
3. 無法分散式：無法在多個伺服器間共享狀態
4. 僅適合開發：不建議用於正式環境

對於正式環境，建議使用：

- SqliteSaver：輕量級資料庫儲存
- PostgresSaver：適合大型應用
- RedisSaver：適合分散式系統

> 未來有興趣再來補這個坑

# 狀態管理進階
## Update State（手動修改狀態）
有時我們需要直接修改 graph 的狀態，而不是透過節點執行。例如：
- 人工介入：管理者修正 AI 回答、刪除不當內容
- 外部事件：Webhook、資料庫更新、第三方 API 回傳
- 系統控制：強制重置計數器、標記流程完成
- Debug / 測試：模擬特定狀態，不用重跑整個 graph

如果只能靠 invoke() 走完整流程，會非常不靈活。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from typing import TypedDict

class ChatState(TypedDict):
    messages: list[dict]
    count: int

def chat_node(state: ChatState):
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": f"這是第 {state['count']} 次回應"}
        ],
        "count": state["count"] + 1
    }

workflow = StateGraph(ChatState)
workflow.add_node("chat", chat_node)
workflow.set_entry_point("chat")
workflow.add_edge("chat", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "chat_001"}}

# 初始執行
state = graph.invoke({
    "messages": [{"role": "user", "content": "你好"}],
    "count": 1
}, config)
print("執行後:", state)


# 手動修改狀態
graph.update_state(config, {
    "messages": state["messages"] + [
        {"role": "system", "content": "系統插入的訊息"}
    ],
    "count": 10  # 直接改成 10
})

# 查看修改後的狀態
current_state = graph.get_state(config)
print("修改後:", current_state.values)


執行後: {'messages': [{'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '這是第 1 次回應'}], 'count': 2}
修改後: {'messages': [{'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '這是第 1 次回應'}, {'role': 'system', 'content': '系統插入的訊息'}], 'count': 10}


## 分支對話（Branching）
Branching 允許從歷史的某個時間點創建新的對話分支，類似 Git 的分支功能。
- 遊戲中的多重結局
- A/B 測試不同的對話策略
- 回溯並嘗試不同的決策路徑
- 保存對話的多個版本

### Time Travel 範例

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from typing import TypedDict

class StoryState(TypedDict):
    story: list[str]
    choice: str

def story_node(state: StoryState):
    new_part = f"選擇了 {state['choice']}，故事繼續..."
    return {
        "story": state["story"] + [new_part],
        "choice": ""
    }

workflow = StateGraph(StoryState)
workflow.add_node("story", story_node)
workflow.set_entry_point("story")
workflow.add_edge("story", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "story_001"}}

# 第一次選擇
state1 = graph.invoke({
    "story": ["從前有個勇者..."],
    "choice": "進入森林"
}, config)
print("選擇1:", state1["story"])

# 第二次選擇
state2 = graph.invoke({
    "story": state1["story"],
    "choice": "遇到巨龍"
}, config)
print("選擇2:", state2["story"])

# 獲取所有歷史 checkpoint
history = list(graph.get_state_history(config))
print(f"\n共有 {len(history)} 個歷史狀態")

# 從第一個選擇點創建分支（回到過去）
first_checkpoint = history[-2]  # 倒數第二個狀態
print(f"\n回到狀態: {first_checkpoint.values['story']}")

# 創建新分支
branch_config = {"configurable": {"thread_id": "story_001_branch"}}
graph.update_state(
    branch_config,
    first_checkpoint.values,
    as_node="story"  # 指定從哪個節點繼續
)

# 在新分支做不同選擇
branch_state = graph.invoke({
    "story": first_checkpoint.values["story"],
    "choice": "返回村莊"
}, branch_config)
print("\n分支選擇:", branch_state["story"])

# 比較兩條路線
print("\n主線:", state2["story"])
print("分支:", branch_state["story"])

選擇1: ['從前有個勇者...', '選擇了 進入森林，故事繼續...']
選擇2: ['從前有個勇者...', '選擇了 進入森林，故事繼續...', '選擇了 遇到巨龍，故事繼續...']

共有 6 個歷史狀態

回到狀態: ['從前有個勇者...']

分支選擇: ['從前有個勇者...', '選擇了 返回村莊，故事繼續...']

主線: ['從前有個勇者...', '選擇了 進入森林，故事繼續...', '選擇了 遇到巨龍，故事繼續...']
分支: ['從前有個勇者...', '選擇了 返回村莊，故事繼續...']


In [ ]:
graph.update_state(
    branch_config,
    first_checkpoint.values,
    as_node="story"
)

- branch_config：新時間線
- values：從過去複製狀態
- as_node：告訴 graph「接下來從哪個節點繼續」